# Puzzle 1 Solution Outline

### What do we know?
* $m_i$ for $m_0, m_1, ..., m_{255}$
* $H(m_i)$ for $m_0, m_1, ..., m_{255}$

$\sum_{i=0}^{256}{a_i * H(m_i)} = a * \sum_{i=0}^{256}H({m_i}) = H(a * \sum_{i=0}^{256}{m_i}) = H(\sum_{i=0}^{256}{a_i * m_i})$


### What do we want to do?

Solve for a, such that:
$$msg = a * \sum_{i=0}^{256}{m_i}$$

Then:
$$H(msg) = a * \sum_{i=0}^{256}{H(m_i)}$$

$a = msg * (\sum_{i=0}^{256}{m_i})^{-1}$

### Steps

* hash my message with blake2s to get a 256-bit integer, which can be modeled as a 256 bit vector (do this in Rust)
* read the blake2s hashes of the messages into a matrix
* multiply msg by inverse of matrix to get $a$
* read the signatures into a matrix
* multiply a * signature matrix to get my hashed signature

In [27]:
# read leaked messages from file into a matrix (write to file using Rust)
A = list()
# bits_vecs-1635288647752 was computed by the rust program, it contains a list of all the messages in bits represantion (after the blake2s hash)
with open("bits_vec-1636574570220", 'r') as f:
    for line_index, line in enumerate(f):
        A.append(list())
        for bit_index in range(0, 256):
            A[line_index].append(int(line[bit_index]))

            
# set up the curve
# define the order of the curve so that we work over the correct field,
# so that the scalars used to solve the matrix will all be in the field
# field is the subgroup size r from here: https://hackmd.io/@benjaminion/bls12-381#Using-curve-BLS12-381
P = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
F = FiniteField(P)

# create the matrix of leaked messages
GA = matrix(F, A)
# need the messages as columns
GAT = GA.transpose()
GAINV = GAT.inverse()
GAINV

# solve for a where aM = msg
# represent msg as a vector
# get the bit representation of the blake2s hash of my message using Rust
msg = vector([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
gsolution = msg * GAINV
gsolution

(...)

In [19]:
# use Rust to represent each leaked message as a bit vector of its blake2s hash
# write all to file "matrix.sage" as a matrix a=matrix([...]) where each message is a row
# load the bit matrix of leaked messages hashes
load('matrix.sage');


# use Rust to represent the blake2s hash of my msg as a bit vector
# write it to file vector.sage as v=vector([...])
# load the msg bit vector
load('vector.sage');

            
# set up the curve
# define the order of the curve so that we work over the correct field,
# so that the scalars used to solve the matrix will all be in the field
# field is the subgroup size r from here: https://hackmd.io/@benjaminion/bls12-381#Using-curve-BLS12-381
P = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
F = FiniteField(P)

# create the matrix of leaked messages on the field F_p
GA = matrix(F, a)
# to represent this as a system of equations, the messages need to be columns in the matrix
# currently they're rows, so transpose the matrix
GAT = GA.transpose()
# now we have GAT * solution_vec = v (bit vector of our hashed message)
# to get solution_vec we need solution_vec = GAT^-1 * v
# get the inverse
GAINV = GAT.inverse()

# solve for the solution vector: GAINV * v
gsolution = GAINV * v

In [20]:
# print the solution vector as Rust code that can be used directly
# each element needs to be read into Arkworks as a scalar field element Fr
",".join('Fr::from_str("' + str(el) + '").unwrap()' for el in gsolution)

# solve for msg_sig = \sum_i{sig[i] * gsolution[i]} in Rust

'Fr::from_str("24054566066078706331491052195577925605048332529656732060838620179268054946396").unwrap(),Fr::from_str("10294910695682276794959842789945945328975765744171699340149303399987732400852").unwrap(),Fr::from_str("25758057617206887245970948775633267322658530058454574059781932453740379028204").unwrap(),Fr::from_str("17008458421943467924844264009652533239726152131365073910754325818893878737120").unwrap(),Fr::from_str("36335715541174884156537351693097990687156376489052229551544870283628363364479").unwrap(),Fr::from_str("38166912488518262594067696882704324422230300872775350309974493889467311474116").unwrap(),Fr::from_str("36644459850513622639580251062048916847472464439573862863256801271055730135673").unwrap(),Fr::from_str("4580177724733811732462204281800958379628611735178655607848589033238842368742").unwrap(),Fr::from_str("13545923753236682429705436960239307239362315630240910154873031795885243243998").unwrap(),Fr::from_str("21655422009463269489661310608218662410249073888481323509918